## Starbucks Filtering

In [163]:
import pandas as pd
import re
import getpass
import json
import numpy as np
import connectionMongoDB as mongo
import getLocation as gl
import foursquare_API as foursq

- Connecting with 1M companies (the mongodb imported with only companies that have raised more tha 1M$

In [4]:
db, coll = mongo.connectCollection('companies','1MCompanies')

In [98]:
company_locations = list(coll.find({},{"Latitude":1,"Longitude":1}))
len(company_locations)

4499

- Using foursquare API to find the nearest starbucks in a radius of 500m for each company office and creating a list of dictionaries with the _id of each office and the lat and long of the nearest starbucks.

In [126]:
all_starbucks=[]
for company in company_locations:
    starbucks=dict()
    comp_lat=company['Latitude']
    comp_long=company['Longitude']
    dat=foursq.apifoursquare(comp_long,comp_lat,"Starbucks",1,500)
    resp=dat['response']
    if resp!={} and resp['totalResults']>0:
        lt=dat['response']['groups'][0]['items'][0]['venue']['location']['lat']
        lg=dat['response']['groups'][0]['items'][0]['venue']['location']['lng']
        starbucks['refcomp']=str(company['_id'])
        starbucks['latitude']=lt
        starbucks['longitude']=lg
        starbucks['geoloc_stb']=gl.getLocation(lg,lt)
        all_starbucks.append(starbucks)

In [153]:
starbucks_df=pd.DataFrame(all_starbucks)

- Exporting Starbucks data to output directory.

In [161]:
# Felipe es genial
with open("starbucks.json", "w+") as file:
    file.write(str(all_starbucks))
#como es string no puedo importarlo a mongo entonces lo edito en code y quito objectid